In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from scipy.stats import chi2_contingency, f_oneway
from category_encoders import TargetEncoder


In [27]:
print(sys.version)

3.9.18 (main, Sep 11 2023, 08:20:50) 
[Clang 14.0.6 ]


In [28]:
import tensorflow as tf

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_selection import mutual_info_classif

In [30]:
df_train = pd.read_csv("GUIDE_Train.csv", nrows=10000)
df_test = pd.read_csv("GUIDE_Test.csv", nrows=10000)

In [31]:
df_train.head(5)

,Id,OrgId,IncidentId,AlertId,Timestamp,DetectorId,AlertTitle,Category,MitreTechniques,IncidentGrade,...,ResourceType,Roles,OSFamily,OSVersion,AntispamDirection,SuspicionLevel,LastVerdict,CountryCode,State,City
0,180388628218,0,612,123247,2024-06-04T06:05:15.000Z,7,6,InitialAccess,NaN,TruePositive,...,NaN,NaN,5,66,NaN,NaN,NaN,31,6,3
1,455266534868,88,326,210035,2024-06-14T03:01:25.000Z,58,43,Exfiltration,NaN,FalsePositive,...,NaN,NaN,5,66,NaN,NaN,NaN,242,1445,10630
2,1056561957389,809,58352,712507,2024-06-13T04:52:55.000Z,423,298,InitialAccess,T1189,FalsePositive,...,NaN,NaN,5,66,NaN,Suspicious,Suspicious,242,1445,10630
3,1279900258736,92,32992,774301,2024-06-10T16:39:36.000Z,2,2,CommandAndControl,NaN,BenignPositive,...,NaN,NaN,5,66,NaN,Suspicious,Suspicious,242,1445,10630
4,214748368522,148,4359,188041,2024-06-15T01:08:07.000Z,9,74,Execution,NaN,TruePositive,...,NaN,NaN,5,66,NaN,NaN,NaN,242,1445,10630


In [32]:
df_train = df_train.drop(columns = ["Id","OrgId","IncidentId","AlertId","DetectorId","DeviceId"])
df_test = df_test.drop(columns = ["Id","OrgId","IncidentId","AlertId","DetectorId","DeviceId"])

In [33]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Timestamp           10000 non-null  object 
 1   AlertTitle          10000 non-null  int64  
 2   Category            10000 non-null  object 
 3   MitreTechniques     4201 non-null   object 
 4   IncidentGrade       9947 non-null   object 
 5   ActionGrouped       59 non-null     object 
 6   ActionGranular      59 non-null     object 
 7   EntityType          10000 non-null  object 
 8   EvidenceRole        10000 non-null  object 
 9   Sha256              10000 non-null  int64  
 10  IpAddress           10000 non-null  int64  
 11  Url                 10000 non-null  int64  
 12  AccountSid          10000 non-null  int64  
 13  AccountUpn          10000 non-null  int64  
 14  AccountObjectId     10000 non-null  int64  
 15  AccountName         10000 non-null  int64  
 16  Devic

In [34]:

#missing_values = df_train.isnull().sum()
#missing_percentages = 100 * missing_values / len(df_train)

# Display columns with missing values
#missing_data = pd.concat([missing_values, missing_percentages], axis=1, keys=['Total', 'Percent'])
#print(missing_data[missing_data['Total'] > 0].sort_values('Percent', ascending=False))


In [35]:
def drop_column_with_over_50_percent_missing(df):
    missing_values = df.isnull().sum()
    missing_percentages = 100 * missing_values / len(df)

    # Display columns with missing values
    missing_data = pd.concat([missing_values, missing_percentages], axis=1, keys=['Total', 'Percent'])
    print(missing_data[missing_data['Total'] > 0].sort_values('Percent', ascending=False))
    
    
    list = missing_data[missing_data.Percent > 50].index.tolist()
    df_drop = df.drop(columns = list)
    return df_drop


df_train_drop = drop_column_with_over_50_percent_missing(df_train)
df_test_drop = drop_column_with_over_50_percent_missing(df_test)

                   Total  Percent
ResourceType        9992    99.92
ActionGrouped       9941    99.41
ActionGranular      9941    99.41
ThreatFamily        9923    99.23
EmailClusterId      9881    98.81
AntispamDirection   9823    98.23
Roles               9789    97.89
SuspicionLevel      8462    84.62
LastVerdict         7600    76.00
MitreTechniques     5799    57.99
IncidentGrade         53     0.53
                   Total  Percent
ActionGrouped       9996    99.96
ActionGranular      9996    99.96
ResourceType        9992    99.92
ThreatFamily        9928    99.28
EmailClusterId      9905    99.05
AntispamDirection   9801    98.01
Roles               9749    97.49
SuspicionLevel      8454    84.54
LastVerdict         7609    76.09
MitreTechniques     5535    55.35


In [36]:
df_train_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Timestamp           10000 non-null  object
 1   AlertTitle          10000 non-null  int64 
 2   Category            10000 non-null  object
 3   IncidentGrade       9947 non-null   object
 4   EntityType          10000 non-null  object
 5   EvidenceRole        10000 non-null  object
 6   Sha256              10000 non-null  int64 
 7   IpAddress           10000 non-null  int64 
 8   Url                 10000 non-null  int64 
 9   AccountSid          10000 non-null  int64 
 10  AccountUpn          10000 non-null  int64 
 11  AccountObjectId     10000 non-null  int64 
 12  AccountName         10000 non-null  int64 
 13  DeviceName          10000 non-null  int64 
 14  NetworkMessageId    10000 non-null  int64 
 15  RegistryKey         10000 non-null  int64 
 16  RegistryValueName   100

In [37]:
def fill_missing_value(df, column):
    df[column].fillna(df[column].mode()[0], inplace = True)

fill_missing_value(df_train_drop, 'IncidentGrade')
fill_missing_value(df_test_drop, 'IncidentGrade')

/var/folders/lf/f0pz1dhd5lv1dtnwnpf8vxt80000gn/T/ipykernel_33047/2763356033.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(df[column].mode()[0], inplace = True)


In [38]:
numerical_columns = df_train_drop.select_dtypes(include=['int64']).columns

In [39]:
cat_columns = df_train_drop.select_dtypes(include=['object']).columns

In [40]:
df_train_drop[numerical_columns].nunique().sort_values(ascending=False)

AccountUpn            3386
AccountName           2323
AccountSid            2211
AlertTitle            2205
AccountObjectId       2200
IpAddress             1879
NetworkMessageId      1193
FileName               706
DeviceName             686
Url                    603
Sha256                 512
FolderPath             465
City                   196
State                  136
CountryCode             68
ApplicationName         23
ApplicationId           21
ResourceIdName          12
RegistryKey             10
OSVersion                6
RegistryValueName        5
RegistryValueData        4
OAuthApplicationId       3
OSFamily                 3
dtype: int64

In [41]:
df_train_drop[cat_columns].nunique().sort_values(ascending=False)

Timestamp        9580
EntityType         20
Category           17
IncidentGrade       3
EvidenceRole        2
dtype: int64

In [42]:
df_train_drop[numerical_columns]

,AlertTitle,Sha256,IpAddress,Url,AccountSid,AccountUpn,AccountObjectId,AccountName,DeviceName,NetworkMessageId,...,ApplicationName,OAuthApplicationId,FileName,FolderPath,ResourceIdName,OSFamily,OSVersion,CountryCode,State,City
0,6,138268,27,160396,441377,673934,425863,453297,153085,529644,...,3421,881,289573,117668,3586,5,66,31,6,3
1,43,138268,360606,160396,22406,23032,22795,24887,153085,529644,...,3421,881,289573,117668,3586,5,66,242,1445,10630
2,298,138268,360606,68652,441377,673934,425863,453297,153085,529644,...,3421,881,289573,117668,3586,5,66,242,1445,10630
3,2,138268,360606,13,441377,673934,425863,453297,153085,529644,...,3421,881,289573,117668,3586,5,66,242,1445,10630
4,74,138268,360606,160396,449,592,440,479,153085,529644,...,3421,881,289573,117668,3586,5,66,242,1445,10630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,138268,360606,160396,441377,673934,425863,453297,153085,529644,...,3421,881,289573,117668,3586,5,66,242,1445,10630
9996,1,138268,360606,160396,45848,53590,43569,41163,153085,529644,...,3421,881,289573,117668,3586,5,66,242,1445,10630
9997,29882,138268,360606,160396,2450,3209,2424,2567,153085,529644,...,3421,881,289573,117668,3586,5,66,242,1445,10630
9998,12372,138268,360606,160396,441377,673934,425863,453297,153085,529644,...,8,881,289573,117668,3586,5,66,242,1445,10630


In [43]:
df_train_drop.columns.unique()

Index(['Timestamp', 'AlertTitle', 'Category', 'IncidentGrade', 'EntityType',
       'EvidenceRole', 'Sha256', 'IpAddress', 'Url', 'AccountSid',
       'AccountUpn', 'AccountObjectId', 'AccountName', 'DeviceName',
       'NetworkMessageId', 'RegistryKey', 'RegistryValueName',
       'RegistryValueData', 'ApplicationId', 'ApplicationName',
       'OAuthApplicationId', 'FileName', 'FolderPath', 'ResourceIdName',
       'OSFamily', 'OSVersion', 'CountryCode', 'State', 'City'],
      dtype='object')

In [44]:
def timestamp_transform(df):
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df['DayOfWeek'] = df['Timestamp'].dt.day_name() 
    df['IsWeekend'] = df['DayOfWeek'].apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)

timestamp_transform(df_train_drop)
timestamp_transform(df_test_drop)

In [45]:
import pandas as pd
from scipy.stats import chi2_contingency, f_oneway
from sklearn.feature_selection import mutual_info_classif

# Significance threshold
alpha = 0.05

# Chi-Square Test for Categorical Features
def chi_square_test(feature, target):
    contingency_table = pd.crosstab(df_train_drop[feature], df_train_drop[target])
    chi2, p, dof, ex = chi2_contingency(contingency_table)
    return p

# ANOVA Test for Numerical Features
def anova_test(feature, target):
    groups = df_train_drop.groupby(target)[feature].apply(list)
    f_stat, p_value = f_oneway(*groups)
    return p_value

# Mutual Information for Numerical Features
def mutual_information(feature, target):
    mi = mutual_info_classif(df_train_drop[[feature]], df_train_drop[target])
    return mi[0]

chi_square_results = {}
anova_results = {}
mutual_info_results = {}

insignificant_features = []

for feature in df_train_drop.columns:
    if feature != 'IncidentGrade':
        if isinstance(df_train_drop[feature].dtype, pd.CategoricalDtype) or pd.api.types.is_object_dtype(df_train_drop[feature]):
            p_value = chi_square_test(feature, 'IncidentGrade')
            chi_square_results[feature] = p_value
            if p_value > alpha:
                insignificant_features.append((feature, 'Chi-Square', p_value))
        elif pd.api.types.is_numeric_dtype(df_train_drop[feature]):
            p_value = anova_test(feature, 'IncidentGrade')
            mi_score = mutual_information(feature, 'IncidentGrade')
            anova_results[feature] = p_value
            mutual_info_results[feature] = mi_score
            if p_value > alpha:
                insignificant_features.append((feature, 'ANOVA', p_value))

# Sort results
sorted_chi_square_results = sorted(chi_square_results.items(), key=lambda item: item[1], reverse=True)
sorted_anova_results = sorted(anova_results.items(), key=lambda item: item[1], reverse=True)
sorted_mi_results = sorted(mutual_info_results.items(), key=lambda item: item[1], reverse=True)

# Print insignificant features
if insignificant_features:
    print("Insignificant Features:")
    for feature, test, p_value in insignificant_features:
        print(f"Feature: {feature}, Test: {test}, P-Value: {p_value}, mutual info: {mi_score}")
else:
    print("All features are significant at the alpha level of", alpha)

# Optionally, print or return the sorted significant results
# print(sorted_chi_square_results, sorted_anova_results, sorted_mi_results)


Insignificant Features:
Feature: RegistryValueName, Test: ANOVA, P-Value: 0.33169492654185023, mutual info: 0.01118486432116983
Feature: RegistryValueData, Test: ANOVA, P-Value: 0.3312493831426669, mutual info: 0.01118486432116983


In [46]:
df_train_significant = df_train_drop.drop(columns = ["RegistryValueName","RegistryValueData"])
df_test_significant = df_test_drop.drop(columns = ["RegistryValueName","RegistryValueData"])

In [47]:
df_train_significant.describe()

,AlertTitle,Sha256,IpAddress,Url,AccountSid,AccountUpn,AccountObjectId,AccountName,DeviceName,NetworkMessageId,...,OAuthApplicationId,FileName,FolderPath,ResourceIdName,OSFamily,OSVersion,CountryCode,State,City,IsWeekend
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,3164.179100,129100.140100,285580.162600,150216.553800,352084.509800,463508.998600,340213.367800,356221.925200,143331.718700,480467.433600,...,880.824000,263596.523000,108249.389000,3582.883900,4.885200,64.484700,224.151900,1353.582200,9952.959800,0.175300
std,11894.681561,33313.245639,141912.774616,37804.399327,166574.325045,290380.115552,160236.388792,174785.603137,35807.555104,141219.984108,...,12.444465,79792.593847,31253.129395,100.103536,0.748517,9.877941,62.023338,347.523692,2580.017748,0.380243
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,138268.000000,360606.000000,160396.000000,441377.000000,90571.500000,425863.000000,441519.000000,153085.000000,529644.000000,...,881.000000,289573.000000,117668.000000,3586.000000,5.000000,66.000000,242.000000,1445.000000,10630.000000,0.000000
50%,11.000000,138268.000000,360606.000000,160396.000000,441377.000000,673934.000000,425863.000000,453297.000000,153085.000000,529644.000000,...,881.000000,289573.000000,117668.000000,3586.000000,5.000000,66.000000,242.000000,1445.000000,10630.000000,0.000000
75%,208.000000,138268.000000,360606.000000,160396.000000,441377.000000,673934.000000,425863.000000,453297.000000,153085.000000,529644.000000,...,881.000000,289573.000000,117668.000000,3586.000000,5.000000,66.000000,242.000000,1445.000000,10630.000000,0.000000
max,109424.000000,138268.000000,360606.000000,160396.000000,441377.000000,673934.000000,425863.000000,453297.000000,153085.000000,529644.000000,...,881.000000,289573.000000,117668.000000,3586.000000,5.000000,66.000000,242.000000,1445.000000,10630.000000,1.000000


Encoding

In [48]:
df_train_significant.nunique().sort_values(ascending=False)

Timestamp             9580
AccountUpn            3386
AccountName           2323
AccountSid            2211
AlertTitle            2205
AccountObjectId       2200
IpAddress             1879
NetworkMessageId      1193
FileName               706
DeviceName             686
Url                    603
Sha256                 512
FolderPath             465
City                   196
State                  136
CountryCode             68
ApplicationName         23
ApplicationId           21
EntityType              20
Category                17
ResourceIdName          12
RegistryKey             10
DayOfWeek                7
OSVersion                6
OAuthApplicationId       3
OSFamily                 3
IncidentGrade            3
EvidenceRole             2
IsWeekend                2
dtype: int64

In [49]:
def target_encode(df, n_bins):
    """
    Preprocesses the data by performing target encoding, frequency encoding, 
    and hashing for high cardinality columns.
    
    Parameters:
    - df (pd.DataFrame): The input DataFrame to preprocess.
    - low_cardinality_columns (list): List of columns with low cardinality for frequency encoding.
    - high_cardinality_columns (list): List of columns with high cardinality for hashing.
    - target_column (str): The name of the target variable for target encoding.
    - n_bins (int, optional): Number of bins for hashing high cardinality columns. Default is 100.
    
    Returns:
    - pd.DataFrame: The preprocessed DataFrame.
    """
    
    low_cardinality_columns = df.nunique()[df.nunique() <= 10].index
    high_cardinality_columns = df.drop(columns = ["Timestamp"]).nunique()[df.nunique() > 10].index

    # Copy the DataFrame to avoid altering the original data
    df_encoded = df.copy()
    
    # Drop the 'Timestamp' column
    if 'Timestamp' in df_encoded.columns:
        df_encoded = df_encoded.drop('Timestamp', axis=1)
    
    # Target encoding
    encoder = TargetEncoder()
    
    # Perform frequency encoding for low cardinality columns
    for column in low_cardinality_columns:
        df_encoded[column] = df_encoded[column].map(df_encoded[column].value_counts())
    
    # Perform hashing for high cardinality columns
    for column in high_cardinality_columns:
        df_encoded[column] = df_encoded[column].apply(lambda x: np.abs(hash(str(x))) % n_bins)
    
    return df_encoded

In [50]:
df_train_encoded = target_encode(df_train_significant, 100)
df_test_encoded = target_encode(df_test_significant, 100)

In [51]:
df_train_encoded.nunique().sort_values(ascending=False)

AlertTitle            100
Url                   100
FileName              100
NetworkMessageId      100
AccountObjectId       100
AccountUpn            100
AccountSid            100
AccountName           100
IpAddress             100
Sha256                 99
DeviceName             99
FolderPath             99
City                   83
State                  68
CountryCode            43
ApplicationName        20
ApplicationId          19
EntityType             18
Category               15
ResourceIdName         12
DayOfWeek               7
OSVersion               4
RegistryKey             3
OSFamily                3
IncidentGrade           3
OAuthApplicationId      2
EvidenceRole            2
IsWeekend               2
dtype: int64

# Model Selection